# Fraud Detection in Python

- 분석 목표 : fraudulent behavior 부정의심거래 탐지 모델 구현 및 Imbalanced Classification
1. Supervised Learning 지도학습으로 부정의심거래 탐지
1. Unsupervised Learning 비지도학습으로 새로운 유형의 부정의심거래 발견
1. 불균형 데이터 분류 문제 핸들링


## module

In [ ]:
!pip install 

In [2]:
# Warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Data Mining
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
from pprint import pprint as pp
import csv
from pathlib import Path
import seaborn as sns
from itertools import product
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import Pipeline 

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score, classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from sklearn.metrics import homogeneity_score, silhouette_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import MiniBatchKMeans, DBSCAN

import gensim
from gensim import corpora

# pandas configuration Options
pd.set_option('display.max_columns', 700)
pd.set_option('display.max_rows', 400)
pd.set_option('display.min_rows', 10)
pd.set_option('display.expand_frame_repr', True)

ModuleNotFoundError: No module named 'imblearn'